In [110]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import gym
import numpy
import onnx
import onnxruntime
from gym import wrappers
import os
from typing import Any, Dict, List, Optional
import glob
# from base.rl.ppo import PPO

In [111]:
pthfile = 'ckpt.test.pth'
onnxpath = 'test_conv_pool.onnx'
print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

1.4.0
1.10.1
1.9.0


In [200]:
def crop_map(h, crop_size):
    
    bs, ch, H, W = h.size()
    bs = torch.tensor(1)
    ch = torch.tensor(4)
    H = torch.tensor(481)
    W = torch.tensor(481)
    
    map_tmp = torch.zeros(1, 3, H+crop_size, W+crop_size)
    map_c1 = torch.ones(1, 1, H+crop_size, W+crop_size)
    map_tmp = torch.cat((map_c1, map_tmp), dim=1)   # (1,4,721,721)
    
    for i in range(ch):
        i = torch.tensor(i)
        map_tmp[0][i][crop_size//2:crop_size//2+H, crop_size//2:crop_size//2+W] = h[0][i]
        
    x_pos = torch.nonzero(map_tmp[0][3]==1)[:][:,0][0]
    y_pos = torch.nonzero(map_tmp[0][3]==1)[:][:,1][0]
    
    
    # print(map_tmp)
    print("x_pos:{}, y_pos:{}".format(x_pos,y_pos))
    # print(map_tmp.size())
    # print(torch.nonzero(map_tmp[0][3]==1))
    
    output = torch.randn(1, 4, crop_size, crop_size)
    
    for i in range(ch):
        # if crop_size%2 == 0:
        output[0][i] = map_tmp[0][i][x_pos-crop_size//2:x_pos+crop_size//2, y_pos-crop_size//2:y_pos+crop_size//2]
        # else:
            # output[0][i] = map_tmp[0][i][x_pos-crop_size//2:x_pos+crop_size//2+1, y_pos-crop_size//2:y_pos+crop_size//2+1]
            
    return output

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self, G):
        super().__init__()
        self.G = torch.tensor(G)
        self.actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            # nn.BatchNorm2d(1),
            Flatten(),  # (G*G, )
            # nn.Sigmoid(),  # added for non-negative
        )
        
        
    def _get_h12(self, inputs): # inputs needs to be a tensor, i.e., original inputs["map_at_t"], (bs, 4, M, M), channel 3 means one-hot pose, channel 0~1 means global map
        # x = inputs["pose_in_map_at_t"]  # (bs,2)
        # map_at_t (4, m, m)
        # x = torch.nonzero(inputs[0][3]==1)
        h = inputs
        
        h_1 = crop_map(h, self.G)
        print(torch.nonzero(h_1[0][3]==1))
        h_2 = F.max_pool2d(h, (2, 2))
# adaptive_max_pool2d
        h_12 = torch.cat([h_1, h_2], dim=1)

        return h_12

    def forward(self, inputs):
        x1 = self._get_h12(inputs)
        x2 = self.actor(x1)
        return x2

In [201]:
actor_net = Global_Actor(240)
actor_net.actor.load_state_dict(torch.load(pthfile, map_location='cpu'),strict=False)
    # Initialize model with the pretrained weights

_IncompatibleKeys(missing_keys=['0.weight', '0.bias', '1.weight', '1.bias', '1.running_mean', '1.running_var', '3.weight', '3.bias', '4.weight', '4.bias', '4.running_mean', '4.running_var', '6.weight', '6.bias', '7.weight', '7.bias', '7.running_mean', '7.running_var', '9.weight', '9.bias', '10.weight', '10.bias', '10.running_mean', '10.running_var', '12.weight', '12.bias'], unexpected_keys=['global_state_dict', 'extra_state'])

In [202]:
inputs = torch.randn(1, 4, 481, 481)

map3 = torch.zeros(1, 481, 481)
map3[0][20][20] = 1.
inputs[0][3] = map3
'''
print(map3)
print(torch.nonzero(map3==1))
inputs[0][3] = map3
yy = torch.nonzero(inputs[0][3]==1)
print(type(yy))
print(type(yy[:,:2]))
print(yy == yy[:,:2])
print(x.size())
'''

x = inputs

In [203]:
def test():
    M = 481
    model = Global_Actor(240)
    batch_size = 1
    input_shape = (4, M, M)
    
    # Initialize model with the pretrained weights
    # map_location = 'gpu' if torch.cuda.is_available() else 'cpu'
    loaded_model = torch.load(pthfile, map_location="cpu")
    model.actor.load_state_dict(loaded_model, strict=False)
    
    # set the model to inference mode
    model.eval()
    
    # data type nchw
    # x = torch.rand(batch_size, *input_shape)
    input_names = ['input']
    output_names = ['output']
    # # Export the model
    torch.onnx.export(model,               # model being run
                    x,                         # model input (or a tuple for multiple inputs)
                    onnxpath,   # where to save the model (can be a file or file-like object)
                    export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=10,          # the ONNX version to export the model to
                    do_constant_folding=True,  # whether to execute constant folding for optimization
                    input_names = ['input'],   # the model's input names
                    output_names = ['output'], # the model's output names
                    dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                  'output' : {0 : 'batch_size'}}
                     )

In [204]:
test()

x_pos:140, y_pos:140
tensor([[120, 120]])


/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:72: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  after removing the cwd from sys.path.
/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignor

In [148]:
a = torch.randn(1,3,2,2)
b = torch.randn(1,1,2,2)
print(a)
print(b)
c = torch.cat((a,b),dim=1)
print(c)
print(c.size())

tensor([[[[-1.1109, -0.9178],
          [-0.5196, -0.7122]],

         [[-0.7214,  0.8115],
          [-0.2302,  0.4412]],

         [[-1.2322, -0.3478],
          [-0.5297,  0.9830]]]])
tensor([[[[ 0.4916, -1.1484],
          [ 0.4830, -1.6397]]]])
tensor([[[[-1.1109, -0.9178],
          [-0.5196, -0.7122]],

         [[-0.7214,  0.8115],
          [-0.2302,  0.4412]],

         [[-1.2322, -0.3478],
          [-0.5297,  0.9830]],

         [[ 0.4916, -1.1484],
          [ 0.4830, -1.6397]]]])
torch.Size([1, 4, 2, 2])
